## Skeleton_extract_for_LSTM

In [6]:
import cv2
import numpy as np
import os 
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def mediapipe_detection(frame, model):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
        
    # Make detection
    results = model.process(image)
        
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
def extract_pose(results):
    # cctv 여서 zero로 처리해도 괜찮을듯 싶다. 아니면 전 데이터를 그대로 가져와야 한다.
    if results.pose_landmarks:
        if results.pose_landmarks.landmark:
            pose = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark]).flatten()
        else:
            pose = np.zeros([0,0])
    else:
        pose = np.zeros(66)
        
    return pose

def count_mp4_files(directory):
    count = 0
    for file in os.listdir(directory):
        if file.endswith(".mp4"):
            count += 1
    return count

def load_video(n):
    original_path = f"/home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/video{n}.mp4"
    augmented_path = f"/home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/augmented_video{n}.mp4"
    
    # 원본 비디오 파일 로드
    cap = cv2.VideoCapture(original_path)
    
    if not cap.isOpened():
        print(f"Failed to open video file: {original_path}")
        return None

    # 비디오 파일의 총 프레임 수 확인
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # 프레임 수가 300보다 낮으면 대체 비디오 파일 로드
    if frame_count < 300:
        cap.release()
        cap = cv2.VideoCapture(augmented_path)
        if not cap.isOpened():
            print(f"Failed to open augmented video file: {augmented_path}")
            return None

    return cap


directory_path = "/home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/"
mp4_count = count_mp4_files(directory_path)

data_path = './test_data_path/break' # path to save .npy
actions = np.array(['breaking']) # action labeling
no_sequence = 10  # Number of sequences
sequence_length = 30  # Length of each sequence

# Create data directory if it doesn't exist
os.makedirs(data_path, exist_ok=True)

# Open video capture
for n in range(1,mp4_count):
    cap = load_video(n)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        for action in actions:
            action_path = os.path.join(data_path, f"video{n}")  # Modify path here
            os.makedirs(action_path, exist_ok=True)
            for sequence in range(no_sequence):
                sequence_path = os.path.join(action_path, f"s{sequence}")
                os.makedirs(sequence_path, exist_ok=True)
                for frame_num in range(sequence_length):
                    ret, frame = cap.read()
                    if not ret:
                        print("Failed to capture frame")
                        break

                    image, results = mediapipe_detection(frame, pose)
                    draw_landmarks(image, results)
                    
                    keypoints = extract_pose(results)
                    npy_path = os.path.join(sequence_path, f"{frame_num}.npy")
                    np.save(npy_path, keypoints)
                    
                    cv2.imshow('Your video', image)
                    
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        cv2.destroyAllWindows()
                        break

    cap.release()

    cv2.destroyAllWindows()


I0000 00:00:1716783324.524190   79696 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1716783324.525393   80214 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1716783324.602710   80203 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716783324.632674   80207 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1716783339.805358   79696 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1716783339.806349   80252 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1716783339.880712   80240 inference_feedback_manager.cc:114] Feedback 

## modifiy video names

In [4]:
import os
import glob

def rename_mp4_files(directory):
    # 디렉토리 내의 모든 mp4 파일 목록 가져오기
    file_paths = glob.glob(os.path.join(directory, "*.mp4"))
    
    # 각 파일 이름을 video1.mp4, video2.mp4, ... 형식으로 변경
    for i, file_path in enumerate(file_paths, start=1):
        # 디렉토리와 파일 확장자 분리
        _, original_filename = os.path.split(file_path)
        filename, extension = os.path.splitext(original_filename)
        
        # 새 파일 이름 생성
        new_filename = f"video{i}{extension}"
        new_file_path = os.path.join(directory, new_filename)
        
        # 파일 이름 변경
        os.rename(file_path, new_file_path)
        print(f'Renamed: {file_path} -> {new_file_path}')

# 대상 디렉토리 경로
directory_path = '/home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우'

# 파일 이름 변경 함수 호출
rename_mp4_files(directory_path)


Renamed: /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/C_5_1_smp_cl_08-29_14-56-00_c_aft_DF1.mp4 -> /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/video1.mp4
Renamed: /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/C_5_1_smp_cl_08-29_15-06-00_c_aft_DF1.mp4 -> /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/video2.mp4
Renamed: /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/C_5_1_smp_cl_08-29_15-25-00_b_aft_DF1.mp4 -> /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/video3.mp4
Renamed: /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/C_5_1_smp_cl_08-29_15-16-00_a_aft_DF1.mp4 -> /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/video4.mp4
Renamed: /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/C_5_1_smp_cl_08-29_14-56-00_b_aft_DF1.mp4 -> /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/video5.mp4
Renamed: /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/C_5_1_smp_cl_08-29_15-16-00_b_aft_DF1.mp4 -> /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/video6.mp4
Renamed: /home/jinsa/Downloads/5-1공원의 안내문을 훼손하는 경우/C_5_1_smp_cl_08-29_15-06-00_b_aft_DF1.mp4 -> /home/jins